In [ ]:
# =====================================================
# BASELINE BALANCE TABLES (Separate: Malawi & Uganda)
# =====================================================

import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats.mstats import winsorize

# =====================================================
# 1️⃣ Load Baseline Data
# =====================================================
malawi_dir = "/Users/arris/Downloads/116346-V1/20160597_addmaterials/Data/Data_Malawi"
uganda_dir = "/Users/arris/Downloads/116346-V1/20160597_addmaterials/Data/Data_Uganda"

# Malawi baseline + treatment
malawi_base = pd.read_stata(f"{malawi_dir}/glasem_baseline.dta", convert_categoricals=False)
malawi_treat = pd.read_stata(f"{malawi_dir}/glasem_treatment.dta", convert_categoricals=False)

# Uganda baseline + marketing
uganda_base = pd.read_stata(f"{uganda_dir}/glaseu_baseline.dta", convert_categoricals=False)
uganda_mark = pd.read_stata(f"{uganda_dir}/glaseu_marketing.dta", convert_categoricals=False)

# Ensure IDs numeric
for df in [malawi_base, malawi_treat, uganda_base, uganda_mark]:
    if "hhid" in df.columns:
        df["hhid"] = pd.to_numeric(df["hhid"], errors="coerce").astype("Int64")

# =====================================================
# 2️⃣ Define Treatment Indicators
# =====================================================
# Malawi: treated if appears in treatment file
malawi_base["treated"] = malawi_base["hhid"].isin(malawi_treat["hhid"]).astype(int)
malawi_base["country"] = 0

# Uganda: treated if voucher given
uganda = uganda_base.merge(uganda_mark, on="hhid", how="left")
uganda["treated"] = (uganda["vouchergivento"] == 1).astype(int)
uganda["country"] = 1

# =====================================================
# 3️⃣ Convert Asset Value to USD and Winsorize (1%)
# =====================================================
for df, label in [(malawi_base, "Malawi"), (uganda, "Uganda")]:
    if "b_assets_value" not in df.columns:
        print(f"⚠️ {label}: no baseline asset variable found, setting to NaN.")
        df["b_assets_value_usd"] = np.nan
        df["b_assets_value_w"] = np.nan
        continue

    # Convert local currency → USD
    if label == "Malawi":
        df["b_assets_value_usd"] = df["b_assets_value"] / 150   # 150 MWK = 1 USD
    elif label == "Uganda":
        df["b_assets_value_usd"] = df["b_assets_value"] / 2290  # 2 290 UGX = 1 USD
    else:
        df["b_assets_value_usd"] = df["b_assets_value"]

    # Winsorize USD value at 1 % tails
    df["b_assets_value_w"] = winsorize(df["b_assets_value_usd"], limits=[0.01, 0.01])

    print(f"✅ {label}: converted and winsorized asset values (in 2010 USD).")


# =====================================================
# 4️⃣ Helper Function: Balance Table
# =====================================================
def balance_table(df, var_map, treat_col="treated", id_col="hhid"):
    results = []
    df_hh = df.groupby([id_col, treat_col], as_index=False).first()

    for var, label in var_map.items():
        if var not in df_hh.columns:
            print(f"⚠️ Skipping {var} (not found)")
            continue

        control = df_hh.loc[df_hh[treat_col] == 0, var].dropna()
        treat   = df_hh.loc[df_hh[treat_col] == 1, var].dropna()

        if control.empty or treat.empty:
            continue

        # Control summary
        cm, csd = control.mean(), control.std()
        cmin, cmax = control.min(), control.max()
        tmin, tmax = treat.min(), treat.max()

        # OLS difference (T−C)
        X = sm.add_constant(df_hh[treat_col])
        y = df_hh[var]
        model = sm.OLS(y, X, missing="drop").fit(cov_type="HC1")
        diff, se = model.params[treat_col], model.bse[treat_col]

        n = df_hh.loc[df_hh[var].notna(), id_col].nunique()

        results.append([
            label, cm, csd, cmin, cmax, tmin, tmax, diff, se, n
        ])

    return pd.DataFrame(results, columns=[
        "Variable", "Control Mean", "Control SD",
        "Control Min", "Control Max", "Treat Min", "Treat Max",
        "Diff (T-C)", "SE", "N (HHs)"
    ])

# =====================================================
# 5️⃣ Variable Map — Baseline Covariates Only
# =====================================================
var_map_balance = {
    "b_age_respondent": "Age of respondent",
    "b_female_resp": "Female respondent",
    "b_n_members": "Household size",
    "b_eduyears": "Years of education",
    "b_assets_value_w": "Asset value (USD, winsorized)",
    "b_n_earner": "Number of earners",
    "b_n_sick": "Number sick household members"
}

# =====================================================
# 6️⃣ Run Separate Balance Tables
# =====================================================
print("=== MALAWI: Baseline Balance (Winsorized Assets) ===\n")
balance_mw = balance_table(malawi_base, var_map_balance)
print(balance_mw.to_string(index=False))

print("\n=== UGANDA: Baseline Balance (Winsorized Assets) ===\n")
balance_ug = balance_table(uganda, var_map_balance)
print(balance_ug.to_string(index=False))

# =====================================================
# 7️⃣ Export to Journal-Style LaTeX Tables (Malawi & Uganda)
# =====================================================

def to_latex_summary(balance_df, caption, fstat, pval, n_obs):
    """
    Convert balance DataFrame + F-test into a LaTeX journal-style summary table.
    """
    latex = rf"""\begin{{table}}[!htbp] \centering
  \caption{{{caption}}}
\begin{{tabular}}{{@{{\extracolsep{{5pt}}}}lcccc}}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
 & \multicolumn{{1}}{{c}}{{Control Mean}} & \multicolumn{{1}}{{c}}{{SD}} & \multicolumn{{1}}{{c}}{{Diff (T--C)}} & \multicolumn{{1}}{{c}}{{SE}} \\
\hline \\[-1.8ex]
"""
    for _, row in balance_df.iterrows():
        latex += (
            f"{row['Variable']} & "
            f"{row['Control Mean']:.2f} & "
            f"{row['Control SD']:.2f} & "
            f"{row['Diff (T-C)']:.3f} & "
            f"({row['SE']:.3f}) \\\\\n"
        )

    latex += rf"""\hline \\[-1.8ex]
Observations & \multicolumn{{4}}{{c}}{{{int(n_obs)} households}} \\
Joint F-test of covariate equality & \multicolumn{{4}}{{c}}{{${{F}}(7, N-8) = {fstat:.2f}$, $p = {pval:.3f}$}} \\
\hline
\hline \\[-1.8ex]
\textit{{Note:}} & \multicolumn{{4}}{{r}}{{Values are means and standard deviations for baseline covariates.}} \\
 & \multicolumn{{4}}{{r}}{{Standard errors (in parentheses) are robust to heteroskedasticity.}} \\
\end{{tabular}}
\end{{table}}
"""
    return latex


# =====================================================
# Compute F-test values beforehand (already done)
# =====================================================
# Example placeholders — replace with your actual computed values
f_mw, p_mw = 1.11, 0.291
f_ug, p_ug = 0.10, 0.749

# N from your screenshot
n_mw, n_ug = 2247, 2160

# =====================================================
# Export journal-style LaTeX
# =====================================================
with open("baseline_summary_malawi.tex", "w") as f:
    f.write(to_latex_summary(balance_mw, "Baseline Summary Statistics and Balance Test – Malawi (Winsorized Assets)",
                             f_mw, p_mw, n_mw))

with open("baseline_summary_uganda.tex", "w") as f:
    f.write(to_latex_summary(balance_ug, "Baseline Summary Statistics and Balance Test – Uganda (Winsorized Assets)",
                             f_ug, p_ug, n_ug))

print("\n✅ Exported:")
print(" - baseline_summary_malawi.tex")
print(" - baseline_summary_uganda.tex")

In [ ]:
#balance test: f Test
from statsmodels.stats.anova import anova_lm

def joint_balance_test(df, covariates, treat_col="treated"):
    """
    Performs a joint F-test for baseline balance:
    H0: All covariate coefficients jointly equal 0 in regression of treatment on covariates.
    """
    df_cov = df[[treat_col] + covariates].dropna()
    y = df_cov[treat_col]
    X = sm.add_constant(df_cov[covariates])
    model = sm.OLS(y, X).fit()

    f_test = model.f_test(" + ".join(covariates) + " = 0")
    return float(f_test.fvalue), float(f_test.pvalue)


# =====================================================
# Run joint balance test (Malawi)
# =====================================================
malawi_covars = [
    "b_age_respondent",
    "b_female_resp",
    "b_n_members",
    "b_eduyears",
    "b_assets_value_w",   # ✅ now in USD and winsorized
    "b_n_earner",
    "b_n_sick"
]
fstat, pval = joint_balance_test(malawi_base, malawi_covars)
print(f"\nJoint balance test (Malawi): F = {fstat:.2f}, p = {pval:.3f}")

# =====================================================
# Run joint balance test (Uganda)
# =====================================================
uganda_covars = [
    "b_age_respondent",
    "b_female_resp",
    "b_n_members",
    "b_eduyears",
    "b_assets_value_w",   # ✅ same adjustment
    "b_n_earner",
    "b_n_sick"
]
fstat, pval = joint_balance_test(uganda, uganda_covars)
print(f"Joint balance test (Uganda): F = {fstat:.2f}, p = {pval:.3f}")


In [ ]:
# =====================================================
# Standardized Differences (Love Plot) – Baseline Balance
# =====================================================

import matplotlib.pyplot as plt
import numpy as np

def plot_standardized_diffs(df, covariates, treat_col="treated", labels=None,
                            title="Covariate Balance", save_path=None):
    """
    Creates and optionally saves a Love plot (standardized differences)
    to visualize baseline balance between treatment and control groups.
    """
    if labels is None:
        labels = covariates

    diffs = []
    valid_labels = []

    for var, lab in zip(covariates, labels):
        if var not in df.columns:
            continue

        control = df.loc[df[treat_col] == 0, var].dropna()
        treat   = df.loc[df[treat_col] == 1, var].dropna()

        if control.empty or treat.empty:
            continue

        diff = treat.mean() - control.mean()
        pooled_sd = np.sqrt((control.var() + treat.var()) / 2)
        diffs.append(diff / pooled_sd)
        valid_labels.append(lab)

    # --- Plot ---
    fig, ax = plt.subplots(figsize=(7, len(diffs)*0.45 + 2))
    y = np.arange(len(diffs))

    ax.axvline(x=0, color="black", linestyle="--", linewidth=1)
    ax.axvline(x=0.1, color="gray", linestyle="--", linewidth=0.8)
    ax.axvline(x=-0.1, color="gray", linestyle="--", linewidth=0.8)

    ax.scatter(diffs, y, c="steelblue", s=60, alpha=0.8, edgecolors="k")
    ax.set_yticks(y)
    ax.set_yticklabels(valid_labels, fontsize=10)
    ax.set_xlabel("Standardized difference (Treatment − Control)", fontsize=11)
    ax.set_title(title, fontsize=13, weight="bold")
    ax.invert_yaxis()

    plt.tight_layout()
    if save_path:
        fig.savefig(save_path, bbox_inches="tight")
        print(f"✅ Saved plot to {save_path}")
    plt.show()


# =====================================================
# 1️⃣ Define Covariates for Each Country
# =====================================================
malawi_covars = [
    "b_age_respondent",
    "b_female_resp",
    "b_n_members",
    "b_eduyears",
    "b_assets_value_w",   # ✅ winsorized, converted to USD
    "b_n_earner",
    "b_n_sick"
]

uganda_covars = malawi_covars.copy()

# =====================================================
# 2️⃣ Generate Balance Plots
# =====================================================

# ---------- Uganda Plot ----------
plot_standardized_diffs(
    uganda,
    uganda_covars,
    treat_col="treated",
    labels=[var_map_balance.get(v, v) for v in uganda_covars],
    title="Baseline Covariate Balance: Uganda",
    save_path="uganda_balance_plot.pdf"
)

# ---------- Malawi Plot ----------
plot_standardized_diffs(
    malawi_base,
    malawi_covars,
    treat_col="treated",
    labels=[var_map_balance.get(v, v) for v in malawi_covars],
    title="Baseline Covariate Balance: Malawi",
    save_path="malawi_balance_plot.pdf"
)

In [ ]:
# =====================================================
# Step1 :MALAWI: Baseline + Treatment + Four-round merge
# =====================================================

print("\n=== MALAWI: Loading and merging ===")

malawi_baseline   = pd.read_stata(f"{malawi_dir}/glasem_baseline.dta", convert_categoricals=False)
malawi_fourrounds = pd.read_stata(f"{malawi_dir}/glasem_four_rounds.dta", convert_categoricals=False)
malawi_treatment  = pd.read_stata(f"{malawi_dir}/glasem_treatment.dta", convert_categoricals=False)

# Convert hhid to numeric
for df in [malawi_baseline, malawi_fourrounds, malawi_treatment]:
    if "hhid" in df.columns:
        df["hhid"] = pd.to_numeric(df["hhid"], errors="coerce").astype("Int64")

# Inspect treatment file
print("Malawi treatment file columns:", malawi_treatment.columns.tolist())
print("Non-missing HHIDs in treatment file:", malawi_treatment["hhid"].notna().sum())

# --- Create 'treated' column only for valid HHIDs ---
malawi_treatment_valid = malawi_treatment.loc[malawi_treatment["hhid"].notna(), ["hhid"]].copy()
malawi_treatment_valid["treated"] = 1
print(f"✅ Treatment file has {len(malawi_treatment_valid)} valid treated households.")

# --- Merge baseline and treatment ---
malawi = malawi_baseline.merge(
    malawi_treatment_valid,
    on="hhid",
    how="left"
)

# --- If 'treated' missing, force create it ---
if "treated" not in malawi.columns:
    print("⚠️ 'treated' column missing after merge — creating treated=0 for all.")
    malawi["treated"] = 0
else:
    malawi["treated"] = malawi["treated"].fillna(0).astype(int)

# --- Merge with four-rounds outcomes ---
malawi = malawi.merge(
    malawi_fourrounds,
    on="hhid",
    how="left"
)

# 🩹 Recover treatment variable
for c in ["treated_x", "treated_y"]:
    if c in malawi.columns:
        malawi.rename(columns={c: "treated"}, inplace=True)
        break

malawi["treated"] = malawi["treated"].fillna(0).astype(int)

print("\n✅ Fixed treatment variable:")
print(malawi["treated"].value_counts(dropna=False))
print("Share treated:", round(malawi["treated"].mean(), 3))

# --------------------------
# Construct household income (Weekly + Monthly)
# --------------------------
weekly_vars = [
    'b_income_business_week_y', 'b_income_labor_week_y', 'b_income_other_week_y',
    'b_income_business_spouse_x', 'b_income_labor_spouse_x', 'b_income_other_spouse_x',
    'b_income_business_weekT_x', 'b_income_labor_weekT_x', 'b_income_other_weekT_x',
    'b_income_business_weekT_y', 'b_income_labor_weekT_y', 'b_income_other_weekT_y'
]

monthly_vars = [
    'b_income_business_month_x', 'b_income_labor_month_x', 'b_income_other_month_x',
    'b_income_business_month_y', 'b_income_labor_month_y', 'b_income_other_month_y',
    'b_income_business_spouse_x', 'b_income_labor_spouse_x', 'b_income_other_spouse_x',
    'b_income_business_monthT_x', 'b_income_labor_monthT_x', 'b_income_other_monthT_x',
    'b_income_business_monthT_y', 'b_income_labor_monthT_y', 'b_income_other_monthT_y'
]

def construct_total_income(df):
    weekly_cols = [col for col in weekly_vars if col in df.columns]
    monthly_cols = [col for col in monthly_vars if col in df.columns]
    weekly = df[weekly_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
    monthly = df[monthly_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
    df["income"] = weekly.sum(axis=1) * 4.3 + monthly.sum(axis=1)
    return df

malawi = construct_total_income(malawi)

print("\n✅ Malawi income summary:")
print(malawi["income"].describe())

# --------------------------
# Clean 'b_assets_value' variable
# --------------------------
if 'b_assets_value_x' in malawi.columns:
    malawi['b_assets_value'] = malawi['b_assets_value_x']
elif 'b_assets_value_y' in malawi.columns:
    malawi['b_assets_value'] = malawi['b_assets_value_y']
elif 'b_assets_dollars' in malawi.columns:
    malawi['b_assets_value'] = malawi['b_assets_dollars']
else:
    print("⚠️ 'b_assets_value' missing in Malawi — setting to 0.")
    malawi['b_assets_value'] = 0

# --------------------------
# Add country indicator for Malawi
# --------------------------
malawi["country"] = 0

In [ ]:
# =====================================================
# STEP 1: UGANDA: Load, Merge, and Clean
# =====================================================

import pandas as pd
import numpy as np

print("=== UGANDA: Loading and merging ===")

# --------------------------
# 1️⃣ Load Uganda files
# --------------------------
uganda_dir = "/Users/arris/Downloads/116346-V1/20160597_addmaterials/Data/Data_Uganda"

uganda_baseline   = pd.read_stata(f"{uganda_dir}/glaseu_baseline.dta", convert_categoricals=False)
uganda_fourrounds = pd.read_stata(f"{uganda_dir}/glaseu_four_rounds.dta", convert_categoricals=False)

# Convert hhid to numeric
for df in [uganda_baseline, uganda_fourrounds]:
    df["hhid"] = pd.to_numeric(df["hhid"], errors="coerce").astype("Int64")

# --------------------------
# 2️⃣ Merge baseline + four rounds
# --------------------------
uganda = uganda_baseline.merge(
    uganda_fourrounds,
    on="hhid",
    how="left"
)

print("✅ Uganda baseline merged with outcomes:", uganda.shape)

# --------------------------
# 3️⃣ Define treatment variable correctly
# --------------------------
# Check which treatment-related columns exist
treat_cols = [c for c in uganda.columns if "treat" in c.lower() or "offer" in c.lower() or "acct" in c.lower()]
print("\nTreatment-related columns found:", treat_cols)

# ✅ Use the randomized assignment variable 'treated_x'
if "treated_x" in uganda.columns:
    uganda["treated"] = uganda["treated_x"].fillna(0).astype(int)
else:
    print("⚠️ No 'treated_x' column found. Setting treated=0 as placeholder.")
    uganda["treated"] = 0

print("\n✅ Uganda treated distribution:")
print(uganda["treated"].value_counts(dropna=False))
print("Share treated:", round(uganda["treated"].mean(), 3))

# --------------------------
# 4️⃣ Create clean 'b_assets_value' variable
# --------------------------
if 'b_assets_value_x' in uganda.columns:
    uganda['b_assets_value'] = uganda['b_assets_value_x']
elif 'b_assets_value_y' in uganda.columns:
    uganda['b_assets_value'] = uganda['b_assets_value_y']
elif 'b_assets_dollars' in uganda.columns:
    uganda['b_assets_value'] = uganda['b_assets_dollars']
else:
    print("⚠️ 'b_assets_value' missing — setting to 0.")
    uganda['b_assets_value'] = 0

# --------------------------
# 5️⃣ Construct household income (weekly + monthly)
# --------------------------
weekly_vars = [
    'b_income_business_week_y', 'b_income_labor_week_y', 'b_income_other_week_y',
    'b_income_business_spouse_x', 'b_income_labor_spouse_x', 'b_income_other_spouse_x'
]
monthly_vars = [
    'b_income_business_month_x', 'b_income_labor_month_x', 'b_income_other_month_x',
    'b_income_business_month_y', 'b_income_labor_month_y', 'b_income_other_month_y'
]

def construct_total_income(df):
    weekly_cols = [col for col in weekly_vars if col in df.columns]
    monthly_cols = [col for col in monthly_vars if col in df.columns]
    weekly = df[weekly_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
    monthly = df[monthly_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
    df["income"] = weekly.sum(axis=1) * 4.3 + monthly.sum(axis=1)
    return df

uganda = construct_total_income(uganda)

print("\n✅ Uganda income summary:")
print(uganda["income"].describe())

# --------------------------
# 6️⃣ Add country indicator
# --------------------------
uganda["country"] = 1

print("\n✅ Final Uganda dataset ready:", uganda.shape)
print(uganda[["hhid", "treated", "b_assets_value", "income"]].head())

In [ ]:
# =====================================================
# ✅ Define post-treatment total savings (correct endline variable) step 1.5
# =====================================================
import numpy as np

for df, label in [(malawi, "Malawi"), (uganda, "Uganda")]:
    if "e_amountsaved_resp_tot" in df.columns:
        df["totsavings"] = df["e_amountsaved_resp_tot"]
        print(f"{label}: using e_amountsaved_resp_tot as post-treatment total savings.")
    elif "mv2_amountsaved_resp_tot" in df.columns:
        df["totsavings"] = df["mv2_amountsaved_resp_tot"]
        print(f"{label}: using mv2_amountsaved_resp_tot (midline fallback).")
    else:
        df["totsavings"] = df.get("b_amountsaved_resp_tot", np.nan)
        print(f"⚠️ {label}: no e_ or mv2_ amountsaved variable found, fallback to baseline.")

# ✅ Check summaries
print("\nMalawi total post-treatment savings summary:")
print(malawi["totsavings"].describe())

print("\nUganda total post-treatment savings summary:")
print(uganda["totsavings"].describe())


In [ ]:
# =====================================================
# 🧹 Filter to only respondents with valid endline data step 1.75
# =====================================================
malawi = malawi.loc[malawi["totsavings"].notna()].copy()
uganda = uganda.loc[uganda["totsavings"].notna()].copy()

print(f"Malawi valid endline obs: {len(malawi)}")
print(f"Uganda valid endline obs: {len(uganda)}")

# ✅ Quick summaries
print("\nMalawi total post-treatment savings summary:")
print(malawi["totsavings"].describe())

print("\nUganda total post-treatment savings summary:")
print(uganda["totsavings"].describe())

In [ ]:
# =====================================================
# STEP 2. Harmonize and Pool Malawi + Uganda
# =====================================================

import pandas as pd

print("=== POOLING MALAWI AND UGANDA ===")

# --------------------------
# 1️⃣ Standardize variable names
# --------------------------
def standardize_columns(df):
    rename_map = {
        'b_age_respondent_x': 'b_age_respondent',
        'b_age_respondent_y': 'b_age_respondent',
        'b_female_resp_x': 'b_female_resp',
        'b_female_resp_y': 'b_female_resp',
        'b_n_members_x': 'b_n_members',
        'b_n_members_y': 'b_n_members',
        'b_eduyears_x': 'b_eduyears',
        'b_eduyears_y': 'b_eduyears'
    }
    return df.rename(columns=rename_map)

malawi = standardize_columns(malawi)
uganda = standardize_columns(uganda)

# --------------------------
# 2️⃣ Keep only common columns
# --------------------------
common_cols = [
    "totsavings",      # outcome 1
    "income",          # outcome 2
    "b_age_respondent",
    "b_female_resp",
    "b_n_members",
    "b_eduyears",
    "b_assets_value",
    "b_n_earner",
    "b_n_sick",
    "treated",
    "country"
]

malawi = malawi[common_cols]
uganda = uganda[common_cols]

# --------------------------
# 3️⃣ Concatenate (stack) the two countries
# --------------------------
pooled = pd.concat([malawi, uganda], ignore_index=True)
pooled = pooled.loc[:, ~pooled.columns.duplicated()]

# Interaction term for heterogeneity
pooled["treated_x_wealth"] = pooled["treated"] * pooled["b_assets_value"]

print("\n✅ Pooled dataset created.")
print("Shape:", pooled.shape)
print("Malawi obs:", len(malawi), "Uganda obs:", len(uganda))
print("\nTreated share (overall):", round(pooled["treated"].mean(), 3))
print(pooled.head())

# =====================================================
# 💵 Convert Malawi + Uganda monetary variables to 2010 USD
# =====================================================

def convert_to_usd(row):
    """
    Convert all monetary variables to 2010 USD using exchange rates
    reported in Dupas et al. (2018):
    1 USD = 2,290 UGX (Uganda)
    1 USD = 150 MWK (Malawi)
    """
    if row["country"] == 1:      # Uganda
        rate = 2290
    else:                        # Malawi
        rate = 150
    return rate

# Apply to key monetary variables
for col in ["totsavings", "income", "b_assets_value"]:
    pooled[f"{col}_usd"] = pooled.apply(lambda r: r[col] / convert_to_usd(r), axis=1)

print("\n✅ Converted all monetary variables to 2010 USD:")
print(pooled[["totsavings_usd", "income_usd", "b_assets_value_usd"]].head())

# Update the interaction term in USD
pooled["treated_x_wealth"] = pooled["treated"] * pooled["b_assets_value_usd"]


In [ ]:
# =====================================================
# STEP 2.5 – Winsorize monetary variables at 1% tails
# =====================================================
def winsorize_series(series, lower=0.01, upper=0.99):
    low, high = series.quantile([lower, upper])
    return series.clip(lower=low, upper=high)

for var in ["totsavings_usd", "income_usd", "b_assets_value_usd"]:
    pooled[var] = winsorize_series(pooled[var])

print("✅ Winsorization applied at 1% tails for savings, income, and assets.")

In [ ]:
# =====================================================
# Step 3. Define Y and X (Post-Treatment Version)
# =====================================================

import numpy as np

# 1️⃣ Define the outcome variable (post-treatment total savings)
Y = pooled["totsavings_usd"].replace([np.inf, -np.inf], np.nan).fillna(0)

# 2️⃣ Define the explanatory variables: treatment, interaction, and controls
X = pooled[[
    "treated",
    "treated_x_wealth",
    "b_age_respondent",
    "b_female_resp",
    "b_n_members",
    "b_eduyears",
    "b_assets_value_usd",
    "b_n_earner",
    "b_n_sick",
    "country"
]].replace([np.inf, -np.inf], np.nan).fillna(0)

# 3️⃣ Rename variables for readability in regression output or export tables
X = X.rename(columns={
    "treated": "Treatment (Bank account)",
    "treated_x_wealth": "Treatment × Wealth",
    "b_age_respondent": "Age of respondent",
    "b_female_resp": "Female respondent",
    "b_n_members": "Household size",
    "b_eduyears": "Years of education",
    "b_assets_value_usd": "Asset value (wealth)",
    "b_n_earner": "Number of earners",
    "b_n_sick": "Number sick household members",
    "country": "Uganda (=1, Malawi=0)"
})

# 4️⃣ Create Numpy versions for use in ML or econometric packages
X_np = X.to_numpy()
Y_np = Y.to_numpy()

# 5️⃣ Quick diagnostic check
print("✅ Y summary (Post-treatment Total Savings):")
print(Y.describe())

print("\n✅ X sample (first 5 rows):")
print(X.head())




In [ ]:
# =====================================================
# STEP 4. LASSO Variable Selection and Regularization Path
# =====================================================

from sklearn import linear_model as skl
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import KFold
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# --------------------------
# 4a. Normalize and compute LASSO path (correct post-treatment version)
# --------------------------

# ✅ Standardize predictors and response
Xs = StandardScaler(with_mean=True, with_std=True).fit_transform(X.fillna(0).to_numpy())
Ys = (Y.fillna(0).to_numpy() - Y.mean()) / Y.std()

# ✅ Compute λmax and λ grid
n = Xs.shape[0]
alpha_max = np.max(np.abs(Xs.T @ Ys)) / n
lambdas = np.logspace(np.log10(alpha_max), np.log10(alpha_max * 1e-4), 100)

# ✅ LASSO path
alphas, soln_array = skl.lasso_path(
    Xs,
    Ys,
    alphas=lambdas,
    max_iter=200000
)[:2]

# Convert to DataFrame for visualization
soln_path = pd.DataFrame(
    soln_array.T,
    columns=X.columns,
    index=-np.log(alphas)
)

# --------------------------
# 4b. Plot coefficient paths
# --------------------------
plt.style.use('seaborn-v0_8-colorblind')
sns.set_style('whitegrid')
sns.set_palette('colorblind')

fig, ax = plt.subplots(figsize=(10, 6))
soln_path.plot(ax=ax, linewidth=1)

ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1),
          title="Predictors", fontsize=9)
ax.set_xlabel(r"$-\log(\lambda)$", fontsize=13)
ax.set_ylabel("Standardized coefficients", fontsize=13)
ax.set_title("LASSO Coefficient Paths (Pooled Malawi + Uganda)", fontsize=15)

plt.tight_layout()
plt.savefig("lasso_coef_paths.pdf", bbox_inches="tight")
plt.show()
plt.close()

# --------------------------
# 4c. Cross-validation for optimal λ
# --------------------------
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

lassoCV = ElasticNetCV(
    alphas=lambdas,
    l1_ratio=1,                 # pure LASSO
    cv=kfold,
    fit_intercept=False,        # already standardized
    random_state=42
)

lassoCV.fit(Xs, Ys)

# Extract CV results
mse_mean = lassoCV.mse_path_.mean(axis=1)
mse_std = lassoCV.mse_path_.std(axis=1)
alphas_cv = lassoCV.alphas_

fig, ax = plt.subplots(figsize=(8, 6))
ax.errorbar(-np.log(alphas_cv), mse_mean, yerr=mse_std,
            fmt="o-", ecolor="gray", capsize=3, markersize=4, linewidth=1)

ax.axvline(-np.log(lassoCV.alpha_), color="red", linestyle="--",
           linewidth=1.5, label="Optimal λ")

ax.set_xlabel(r"$-\log(\lambda)$", fontsize=13)
ax.set_ylabel("Cross-validated MSE", fontsize=13)
ax.set_title("LASSO CV MSE (Pooled Malawi + Uganda)", fontsize=15)
ax.legend()

plt.tight_layout()
plt.savefig("lasso_cv_mse.pdf", bbox_inches="tight")
plt.show()
plt.close()

print(f"✅ Optimal alpha (λ): {lassoCV.alpha_:.6f}")

# ✅ Coefficients at optimal λ
lasso_coefs = pd.Series(lassoCV.coef_, index=X.columns)
print("\nTop 10 nonzero LASSO coefficients:")
print(lasso_coefs[lasso_coefs != 0].sort_values(ascending=False).head(10).round(4))


In [ ]:
# ==============================================================
# STEP 4. Ridge Regularization (Coefficient Path + CV MSE, Improved Visualization)
# ==============================================================

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from sklearn.linear_model import enet_path, ElasticNetCV
from sklearn.model_selection import KFold

# ✅ Apply consistent style (same as LASSO)
plt.style.use('seaborn-v0_8-colorblind')
sns.set_style('whitegrid')

# ==============================================================
# 4a. Ridge Coefficient Path (Improved Visualization)
# ==============================================================

ridge_lambdas = np.logspace(5, -2, 200)  # from 10^5 down to 10^-2

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message=".*Coordinate descent without L1 regularization.*")
    warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.linear_model._coordinate_descent")

    # Compute Ridge coefficient path (l1_ratio=0 → pure Ridge)
    alphas_ridge, soln_array_ridge = enet_path(
        Xs, Ys,
        l1_ratio=0.0,
        alphas=ridge_lambdas,
        max_iter=200000
    )[:2]

# Convert to DataFrame for plotting
soln_path_ridge = pd.DataFrame(
    soln_array_ridge.T,
    columns=X.columns,
    index=-np.log(alphas_ridge)
)

# Improved plot: distinct colors per variable
palette = sns.color_palette("tab10", n_colors=len(X.columns))

fig, ax = plt.subplots(figsize=(10, 6))

for i, col in enumerate(soln_path_ridge.columns):
    ax.plot(
        soln_path_ridge.index,
        soln_path_ridge[col],
        label=col,
        linewidth=1.5,
        color=palette[i % len(palette)]
    )

# Legend and labels
ax.legend(
    loc='upper left',
    bbox_to_anchor=(1.05, 1),
    title="Predictors",
    fontsize=9,
    title_fontsize=10,
    frameon=True
)
ax.set_xlabel(r"$-\log(\lambda)$", fontsize=13)
ax.set_ylabel("Standardized coefficients", fontsize=13)
ax.set_title("Ridge Coefficient Paths (Pooled Malawi + Uganda)", fontsize=15)
ax.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.savefig("ridge_coef_paths.pdf", bbox_inches="tight")
plt.show()
plt.close(fig)

# ==============================================================
# 4b. Ridge Cross-Validation (MSE)
# ==============================================================

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message=".*Coordinate descent without L1 regularization.*")
    warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.linear_model._coordinate_descent")

    ridgeCV = ElasticNetCV(
        alphas=ridge_lambdas,
        l1_ratio=0,             # pure Ridge (no LASSO penalty)
        cv=kfold,
        fit_intercept=False,    # standardized data
        random_state=42
    )
    ridgeCV.fit(Xs, Ys)

# Extract mean & std of MSE across folds
mse_mean_ridge = ridgeCV.mse_path_.mean(axis=1)
mse_std_ridge = ridgeCV.mse_path_.std(axis=1)
alphas_cv_ridge = ridgeCV.alphas_

# Plot Ridge CV MSE vs λ
fig, ax = plt.subplots(figsize=(8, 6))
ax.errorbar(
    -np.log(alphas_cv_ridge),
    mse_mean_ridge,
    yerr=mse_std_ridge,
    fmt="o-",
    ecolor="gray",
    capsize=3,
    markersize=4,
    linewidth=1
)

# Highlight optimal λ
ax.axvline(
    -np.log(ridgeCV.alpha_),
    color="red",
    linestyle="--",
    linewidth=1.5,
    label="Optimal λ"
)

ax.set_xlabel(r"$-\log(\lambda)$", fontsize=13)
ax.set_ylabel("Cross-validated MSE", fontsize=13)
ax.set_title("Ridge CV MSE (Pooled Malawi + Uganda)", fontsize=15)
ax.legend()

plt.tight_layout()
plt.savefig("ridge_cv_mse.pdf", bbox_inches="tight")
plt.show()
plt.close(fig)

# ==============================================================
# 4c. Coefficient Summary
# ==============================================================

print(f"✅ Optimal Ridge alpha (λ): {ridgeCV.alpha_:.6f}")

ridge_coefs = pd.Series(ridgeCV.coef_, index=X.columns)
nonzero_ridge = ridge_coefs[ridge_coefs.abs() > 1e-6]  # filter near-zero

print("\nTop 10 Ridge coefficients (largest in magnitude):")
print(nonzero_ridge.sort_values(ascending=False).head(10).round(4))




In [ ]:
# ==============================================================
# STEP 5. OLS Benchmark — ITT for Savings and Income
# ==============================================================

import statsmodels.api as sm
import pandas as pd
import numpy as np
from stargazer.stargazer import Stargazer  # pretty LaTeX output

# 1️⃣ Add intercept
X_ols = sm.add_constant(X)

# 2️⃣ Define dependent variables
Y_savings = pooled["totsavings_usd"].replace([np.inf, -np.inf], np.nan).fillna(0)
Y_income  = pooled["income_usd"].replace([np.inf, -np.inf], np.nan).fillna(0)

# 3️⃣ Estimate both models (robust HC1 standard errors)
ols_savings = sm.OLS(Y_savings, X_ols).fit(cov_type="HC1")
ols_income  = sm.OLS(Y_income,  X_ols).fit(cov_type="HC1")

# 4️⃣ Print detailed summaries in Python
print("\n=== OLS Results: Total Savings (in 2010 USD) ===")
print(ols_savings.summary())

print("\n=== OLS Results: Total Income (in 2010 USD) ===")
print(ols_income.summary())

# 4️⃣ Combine and export in journal-style LaTeX
stargazer = Stargazer([ols_savings, ols_income])
stargazer.title("OLS Regression: Total Savings and Total Income (ITT)")
stargazer.custom_columns(["(1) Total Savings", "(2) Total Income"], [1, 1])
stargazer.show_adj_r2 = True
stargazer.significant_digits(3)

# 5️⃣ Render LaTeX
latex_code = stargazer.render_latex()
print(latex_code)





In [ ]:
# =====================================================
# STEP 6. Country-specific LASSO & Ridge (Uganda / Malawi)
# =====================================================

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
import sklearn.linear_model as skl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm

# Consistent, clear style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("talk", font_scale=1.2)
sns.set_palette("husl", n_colors=10)  # Distinct hues

# =====================================================
# 🔹 Helper function for country-specific analysis
# =====================================================

def run_lasso_ridge_country(df, country_label):
    print(f"\n=== {country_label.upper()}: LASSO & Ridge ===")

    # 1️⃣ Define Y and X
    Y = df["totsavings_usd"].fillna(0)
    X = df[[
        "treated",
        "treated_x_wealth",
        "b_age_respondent",
        "b_female_resp",
        "b_n_members",
        "b_eduyears",
        "b_assets_value_usd",
        "b_n_earner",
        "b_n_sick"
    ]].fillna(0)

    X = X.rename(columns={
        "treated": "Treatment (Bank account)",
        "treated_x_wealth": "Treatment × Wealth",
        "b_age_respondent": "Age of respondent",
        "b_female_resp": "Female respondent",
        "b_n_members": "Household size",
        "b_eduyears": "Years of education",
        "b_assets_value_usd": "Asset value (wealth)",
        "b_n_earner": "Number of earners",
        "b_n_sick": "Number sick household members"
    })

    # 2️⃣ Standardize predictors and response
    scaler = StandardScaler(with_mean=True, with_std=True)
    Xs = scaler.fit_transform(X)
    Ys = (Y - Y.mean()) / Y.std()

    # 3️⃣ Compute LASSO path
    n = Xs.shape[0]
    alpha_max = np.max(np.abs(Xs.T @ Ys)) / n
    lambdas = np.logspace(np.log10(alpha_max), np.log10(alpha_max * 1e-4), 100)

    alphas, soln_array = skl.lasso_path(Xs, Ys, alphas=lambdas, max_iter=200000)[:2]
    soln_path = pd.DataFrame(soln_array.T, columns=X.columns, index=-np.log(alphas))

    # 4️⃣ Plot LASSO coefficient paths
    fig, ax = plt.subplots(figsize=(12, 7))
    colors = cm.get_cmap('tab10', len(X.columns))  # Use 10 distinct colors

    for i, col in enumerate(soln_path.columns):
        ax.plot(soln_path.index, soln_path[col],
                color=colors(i),
                label=col,
                linewidth=2.0)

    ax.set_xlabel(r"$-\log(\lambda)$", fontsize=14)
    ax.set_ylabel("Standardized coefficients", fontsize=14)
    ax.set_title(f"LASSO Coefficient Paths – {country_label}", fontsize=16, weight='bold')
    ax.legend(title="Predictors", loc="upper left", bbox_to_anchor=(1.05, 1), fontsize=10, title_fontsize=11)
    ax.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(f"lasso_coef_paths_{country_label.lower()}.pdf", bbox_inches="tight", dpi=400)
    plt.show()
    plt.close()

    # 5️⃣ Ridge regression
    ridge_lambdas = np.logspace(-4, 4, 200)
    ridge = RidgeCV(alphas=ridge_lambdas, store_cv_values=True)
    ridge.fit(Xs, Ys)

    print(f"✅ Best Ridge alpha ({country_label}): {ridge.alpha_:.5f}")
    ridge_coefs = pd.Series(ridge.coef_, index=X.columns)
    print(f"\nTop Ridge coefficients ({country_label}):")
    print(ridge_coefs.sort_values(ascending=False).round(4).head(10))

    return ridge_coefs, ridge.alpha_


# =====================================================
# 🔸 Run for Uganda and Malawi
# =====================================================

ridge_coefs_ug, alpha_ug = run_lasso_ridge_country(
    pooled[pooled["country"] == 1],
    "Uganda"
)

ridge_coefs_mw, alpha_mw = run_lasso_ridge_country(
    pooled[pooled["country"] == 0],
    "Malawi"
)


In [ ]:
# =====================================================
# STEP 7. Robustness Check: Log of Total Savings (OLS)
# =====================================================

import numpy as np
import statsmodels.api as sm
from stargazer.stargazer import Stargazer

# ✅ 1. Create log-transformed dependent variable (avoid log(0))
pooled["log_totsavings_usd"] = np.log(pooled["totsavings_usd"] + 1)

# ✅ 2. Define outcome and regressors (using harmonized columns)
Y_log = pooled["log_totsavings_usd"].replace([np.inf, -np.inf], np.nan).fillna(0)

X_log = pooled[[
    "treated",
    "treated_x_wealth",
    "b_age_respondent",
    "b_female_resp",
    "b_n_members",
    "b_eduyears",
    "b_assets_value_usd",
    "b_n_earner",
    "b_n_sick",
    "country"
]].replace([np.inf, -np.inf], np.nan).fillna(0)

# ✅ 3. Rename columns for interpretability
X_log = X_log.rename(columns={
    "treated": "Treatment (Bank account)",
    "treated_x_wealth": "Treatment × Wealth",
    "b_age_respondent": "Age of respondent",
    "b_female_resp": "Female respondent",
    "b_n_members": "Household size",
    "b_eduyears": "Years of education",
    "b_assets_value_usd": "Asset value (wealth)",
    "b_n_earner": "Number of earners",
    "b_n_sick": "Number sick household members",
    "country": "Uganda (=1, Malawi=0)"
})

# ✅ 4. Add constant term and fit OLS (heteroskedasticity-robust)
X_log_ols = sm.add_constant(X_log)
ols_log_model = sm.OLS(Y_log, X_log_ols).fit(cov_type="HC1")

print("=== Robustness Check: Log of Total Savings (OLS) ===")
print(ols_log_model.summary())

# =====================================================
# STEP 8. Export to LaTeX (for Overleaf / Paper Appendix)
# =====================================================

stargazer_log = Stargazer([ols_log_model])
stargazer_log.title("Robustness Check: Log of Total Savings (OLS)")
stargazer_log.custom_columns(["Pooled Sample"], [1])
stargazer_log.show_degrees_of_freedom(False)
stargazer_log.show_model_numbers(False)
stargazer_log.covariate_order([
    "Treatment (Bank account)",
    "Treatment × Wealth",
    "Age of respondent",
    "Female respondent",
    "Household size",
    "Years of education",
    "Asset value (wealth)",
    "Number of earners",
    "Number sick household members",
    "Uganda (=1, Malawi=0)"
])

latex_code = stargazer_log.render_latex()

# ✅ Save LaTeX output
with open("robustness_log_savings.tex", "w") as f:
    f.write(latex_code)

# ✅ Print preview to console
print("\n\n=== LaTeX Table Code ===")
print(latex_code)

In [ ]:
# =====================================================
# Regression Tree (Pooled Malawi + Uganda)
# =====================================================

# --------------------------
# Step 1. Imports
# --------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from matplotlib.pyplot import subplots

# --------------------------
# Step 2. Define Y and X
# --------------------------
Y = pooled["totsavings_usd"].replace([np.inf, -np.inf], np.nan).fillna(0).values.reshape(-1, 1)

X = pooled[[
    "treated",
    "treated_x_wealth",
    "b_age_respondent",
    "b_female_resp",
    "b_n_members",
    "b_eduyears",
    "b_assets_value_usd",
    "b_n_earner",
    "b_n_sick",
    "country"
]].replace([np.inf, -np.inf], np.nan).fillna(0)

X = X.rename(columns={
    "treated": "Treatment (Bank account)",
    "treated_x_wealth": "Treatment × Wealth",
    "b_age_respondent": "Age of respondent",
    "b_female_resp": "Female respondent",
    "b_n_members": "Household size",
    "b_eduyears": "Years of education",
    "b_assets_value_usd": "Asset value (wealth)",
    "b_n_earner": "Number of earners",
    "b_n_sick": "Number sick household members",
    "country": "Uganda (=1, Malawi=0)"
})

# --------------------------
# Step 3. Train-test split
# --------------------------
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42
)

# --------------------------
# Step 4. MinMax Scaling for both X and Y
# --------------------------
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

# Scale X
X_train = pd.DataFrame(
    x_scaler.fit_transform(X_train),
    columns=X.columns,
    index=X_train.index
)
X_test = pd.DataFrame(
    x_scaler.transform(X_test),
    columns=X.columns,
    index=X_test.index
)

# Scale Y
Y_train = y_scaler.fit_transform(Y_train)
Y_test = y_scaler.transform(Y_test)

# --------------------------
# Step 5. Fit Regression Tree
# --------------------------
reg = DecisionTreeRegressor(max_depth=3, random_state=42)
reg.fit(X_train, Y_train)

# --------------------------
# Step 6. Plot Regression Tree
# --------------------------
fig, ax = subplots(figsize=(15,10))
plot_tree(
    reg,
    feature_names=X.columns,
    filled=True,
    rounded=True,
    fontsize=13,          # ✅ Larger text for readability
    precision=3,          # ✅ Show fewer decimals in node values
    proportion=True,      # ✅ Show samples as percentages, not raw counts
    impurity=False,       # ✅ Hide impurity (keeps boxes clean)
    ax=ax
)

ax.set_title(
    "Regression Tree — Pooled Malawi & Uganda (Scaled Features in USD)",
    fontsize=20,
    pad=20,
    weight="bold"
)

plt.tight_layout()
plt.savefig("regression_tree.pdf", bbox_inches="tight", dpi=400)
plt.show()
plt.close(fig)




In [ ]:
# --------------------------
# Step 7. Random Forest Feature Importance
# --------------------------
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns

# ✅ Random Forest (with modest depth for interpretability)
rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=5,
    random_state=42
)

# Fit model (flatten Y_train since it’s 2D after scaling)
rf.fit(X_train, Y_train.ravel())

# --------------------------
# Step 8. Create Feature Importance Table
# --------------------------
importances = rf.feature_importances_
feat_importance = (
    pd.DataFrame({
        "Feature": X.columns,
        "Importance": importances
    })
    .sort_values(by="Importance", ascending=False)
    .reset_index(drop=True)
)

print("=== Random Forest Feature Importances ===")
print(feat_importance.round(4))

# =====================================================
# Step 9. Plot Feature Importance (clean version)
# =====================================================
plt.style.use('seaborn-v0_8-colorblind')
sns.set_style('whitegrid')

fig, ax = plt.subplots(figsize=(9, 5.5))

# Barplot with reversed viridis for lighter color emphasis on top features
sns.barplot(
    data=feat_importance,
    x="Importance",
    y="Feature",
    palette="viridis_r",
    ax=ax,
    linewidth=0
)

# Titles and labels
ax.set_title("Random Forest Feature Importance (Pooled Malawi + Uganda)", fontsize=15, pad=12)
ax.set_xlabel("Feature Importance", fontsize=13)
ax.set_ylabel("")

# Add value labels to the right of bars
for i, v in enumerate(feat_importance["Importance"]):
    ax.text(v + 0.005, i, f"{v:.2f}", color='black', va='center', fontsize=10)

# Aesthetics
ax.xaxis.grid(True, linestyle="--", alpha=0.6)
ax.yaxis.grid(False)
ax.tick_params(axis="x", labelsize=11)
ax.tick_params(axis="y", labelsize=12)
sns.despine(left=True, bottom=True)

plt.tight_layout()
fig.savefig("feature_importance.pdf", bbox_inches="tight")
plt.show()
plt.close(fig)



In [ ]:
# --------------------------
# Step X. Error Comparison: Tree, Bagging, Random Forest, Boosting
# --------------------------
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor as RF, GradientBoostingRegressor as GBR
import numpy as np
import pandas as pd

# Flatten Y to avoid shape issues (since MinMaxScaler made it 2D)
Y_train_flat = Y_train.ravel()
Y_test_flat = Y_test.ravel()

# --------------------------
# 1️⃣ Regression Tree (baseline)
# --------------------------
y_hat_tree = reg.predict(X_test)
mse_tree = mean_squared_error(Y_test_flat, y_hat_tree)

# --------------------------
# 2️⃣ Bagging (RF using all features)
# --------------------------
bagging = RF(
    n_estimators=500,
    max_features=len(X.columns),
    random_state=42,
    bootstrap=True
)
bagging.fit(X_train, Y_train_flat)
y_hat_bag = bagging.predict(X_test)
mse_bag = mean_squared_error(Y_test_flat, y_hat_bag)

# --------------------------
# 3️⃣ Random Forest (sqrt features per split)
# --------------------------
rf = RF(
    n_estimators=500,
    max_features="sqrt",
    random_state=42
)
rf.fit(X_train, Y_train_flat)
y_hat_rf = rf.predict(X_test)
mse_rf = mean_squared_error(Y_test_flat, y_hat_rf)

# --------------------------
# 4️⃣ Gradient Boosting
# --------------------------
boost = GBR(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)
boost.fit(X_train, Y_train_flat)
y_hat_boost = boost.predict(X_test)
mse_boost = mean_squared_error(Y_test_flat, y_hat_boost)

# --------------------------
# 5️⃣ Summarize Errors
# --------------------------
errors = pd.DataFrame({
    "Model": ["Regression Tree", "Bagging", "Random Forest", "Boosting"],
    "MSE": [mse_tree, mse_bag, mse_rf, mse_boost],
    "RMSE": [np.sqrt(mse_tree), np.sqrt(mse_bag), np.sqrt(mse_rf), np.sqrt(mse_boost)]
}).round(6)

print("\n=== Model Error Comparison (Scaled X & Y in USD) ===")
print(errors)

In [ ]:
# --------------------------
# 5️⃣ Summarize Errors (more precision)
# --------------------------
errors = pd.DataFrame({
    "Model": ["Regression Tree", "Bagging", "Random Forest", "Boosting"],
    "MSE": [mse_tree, mse_bag, mse_rf, mse_boost],
    "RMSE": [np.sqrt(mse_tree), np.sqrt(mse_bag), np.sqrt(mse_rf), np.sqrt(mse_boost)]
})

# Show with 6 decimal places for precision
pd.options.display.float_format = "{:.6f}".format

print("\n=== Model Error Comparison (Scaled X & Y in USD) ===")
print(errors)


In [ ]:
# =====================================================
# Export Model Error Comparison to LaTeX
# =====================================================

from stargazer.stargazer import Stargazer

# Create a simple fake "model" container so Stargazer can render it
# (Stargazer expects regression results, so we’ll just format manually)
latex_table = r"""
\begin{table}[htbp]
\centering
\caption{Model Error Comparison (Scaled X and Y)}
\begin{tabular}{lcc}
\hline
\textbf{Model} & \textbf{MSE} & \textbf{RMSE} \\
\hline
"""

for _, row in errors.iterrows():
    latex_table += f"{row['Model']} & {row['MSE']:.6f} & {row['RMSE']:.6f} \\\\\n"

latex_table += r"""\hline
\end{tabular}
\end{table}
"""

# Save LaTeX code to a .tex file
with open("model_error_comparison.tex", "w") as f:
    f.write(latex_table)

print("\n\n=== LaTeX Table Code ===")
print(latex_table)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Helper function
def draw_gml(gml_str, title, outfile, figsize=(10,7), seed=42):
    G = nx.parse_gml(gml_str)
    plt.figure(figsize=figsize)
    pos = nx.spring_layout(G, seed=seed)
    nx.draw(
        G, pos,
        with_labels=True,
        node_size=3500,
        node_color='blue',
        font_size=9,
        font_color='white',
        arrowsize=15
    )
    plt.title(title, fontsize=14)
    plt.axis('off')
    plt.savefig(outfile, dpi=300, bbox_inches='tight')
    plt.show()

# -------------------------------------------------------------------
# 1️⃣ Macro-Level DAG (RCT) with Interaction
# -------------------------------------------------------------------
macro_rct_with_interaction = """graph [
  directed 1

  node [ id 0 label "Country" ]
  node [ id 1 label "Treatment" ]
  node [ id 2 label "Wealth" ]
  node [ id 3 label "Education" ]
  node [ id 4 label "Savings" ]
  node [ id 5 label "T × W" ]

  edge [ source 0 target 1 ]   # Country → Treatment (stratified randomization)
  edge [ source 0 target 2 ]   # Country → Wealth
  edge [ source 0 target 3 ]   # Country → Education
  edge [ source 0 target 4 ]   # Country → Savings
  edge [ source 3 target 2 ]   # Education → Wealth

  edge [ source 1 target 5 ]   # Treatment → (T×W)
  edge [ source 2 target 5 ]   # Wealth → (T×W)
  edge [ source 5 target 4 ]   # (T×W) → Savings

  edge [ source 1 target 4 ]   # Treatment → Savings
  edge [ source 2 target 4 ]   # Wealth → Savings
  edge [ source 3 target 4 ]   # Education → Savings
]"""
G = nx.parse_gml(macro_rct_with_interaction)
plt.figure(figsize=(9,6))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True,
        node_size=3500, node_color='blue',
        font_size=9, font_color='white', arrowsize=15)
plt.title("Macro-Level DAG (RCT) with Interaction", fontsize=14)
plt.axis('off')
plt.savefig("macro_rct_with_interaction.pdf", format="pdf", bbox_inches='tight')
plt.show()

In [ ]:
household_rct_with_interaction = """graph [
  directed 1

  node [ id 0 label "Country" ]
  node [ id 1 label "Treatment" ]
  node [ id 2 label "Wealth" ]
  node [ id 3 label "Education" ]
  node [ id 4 label "Savings" ]
  node [ id 5 label "T × W" ]
  node [ id 6 label "Age" ]
  node [ id 7 label "Female" ]
  node [ id 8 label "HH size" ]
  node [ id 9 label "N-earner" ]
  node [ id 10 label "N-sick" ]

  edge [ source 0 target 1 ]   # Country → Treatment
  edge [ source 0 target 2 ]   # Country → Wealth
  edge [ source 0 target 3 ]   # Country → Education
  edge [ source 0 target 4 ]   # Country → Savings
  edge [ source 3 target 2 ]   # Education → Wealth

  edge [ source 1 target 5 ]   # Treatment → (T×W)
  edge [ source 2 target 5 ]   # Wealth → (T×W)
  edge [ source 5 target 4 ]   # (T×W) → Savings

  edge [ source 1 target 4 ]   # Treatment → Savings
  edge [ source 2 target 4 ]   # Wealth → Savings
  edge [ source 3 target 4 ]   # Education → Savings
  edge [ source 6 target 4 ]   # Age → Savings
  edge [ source 7 target 4 ]   # Female → Savings
  edge [ source 8 target 4 ]   # HH size → Savings
  edge [ source 9 target 4 ]   # N-earner → Savings
  edge [ source 10 target 4 ]  # N-sick → Savings
]"""

G = nx.parse_gml(household_rct_with_interaction)
plt.figure(figsize=(11,7))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True,
        node_size=3500, node_color='blue',
        font_size=9, font_color='white', arrowsize=15)
plt.title("Household-Level DAG (RCT) with Interaction", fontsize=14)
plt.axis('off')
plt.savefig("household_rct_with_interaction.pdf", format="pdf", bbox_inches='tight')
plt.show()

In [ ]:
heterogeneity_rct_tight = """graph [
  directed 1

 node [ id 0 label "Country" ]
  node [ id 1 label "Treatment" ]
  node [ id 2 label "Wealth" ]
  node [ id 3 label "Education" ]
  node [ id 4 label "Savings" ]
  node [ id 5 label "T × W" ]

  edge [ source 0 target 1 ]   # Country → Treatment
  edge [ source 0 target 2 ]   # Country → Wealth
  edge [ source 0 target 3 ]   # Country → Education
  edge [ source 0 target 4 ]   # Country → Savings
  edge [ source 3 target 2 ]   # Education → Wealth

  edge [ source 1 target 5 ]   # Treatment → (T×W)
  edge [ source 2 target 5 ]   # Wealth → (T×W)
  edge [ source 5 target 4 ]   # (T×W) → Savings

  edge [ source 1 target 4 ]   # Treatment → Savings
  edge [ source 2 target 4 ]   # Wealth → Savings
]"""

G = nx.parse_gml(heterogeneity_rct_tight)
plt.figure(figsize=(9,6))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True,
        node_size=3500, node_color='blue',
        font_size=9, font_color='white', arrowsize=15)
plt.title("Heterogeneity DAG (RCT): Treatment × Wealth Interaction", fontsize=14)
plt.axis('off')
plt.savefig("heterogeneity_rct_with_interaction.pdf", format="pdf", bbox_inches='tight')
plt.show()